# MobileCoin Example Wallet

This is an example python client that interacts with `mobilecoind` to manage a MobileCoin wallet.

You must start the `mobilecoind` daemon in order to run a wallet. See the mobilecoind README for more information.

To run this notebook, make sure you have the requirements installed, and that you have compiled the grpc protos.

```
cd mobilecoind/clients/python/jupyter
./install.sh
jupyter notebook
```

In [ ]:
from mobilecoin import Client

#### Start the Mob Client

The client talks to your local mobilecoind. See the mobilecoind/README.md for information on how to set it up.

In [ ]:
client = Client("localhost:4444", ssl=False)

#### Input Root Entropy for Account

Note: The root entropy is sensitive material. It is used as the seed to create your account keys. Anyone with your root entropy can steal your MobileCoin.

In [ ]:
entropy = "4ec2c081e764f4189afba528956c05804a448f55f24cc3d04c9ef7e807a93bcd"
credentials_response = client.get_account_key(bytes.fromhex(entropy))

#### Monitor your Account

Monitoring an account means that mobilecoind will persist the transactions that belong to you to a local database. This allows you to retrieve your funds and calculate your balance, as well as to construct and submit transactions.

Note: MobileCoin uses accounts and subaddresses for managing funds. You can optionally specify a range of subaddresses to monitor. See mob_client.py for more information.

In [ ]:
monitor_id_response = client.add_monitor(credentials_response.account_key)

#### Check Balance

You will need to provide a subaddress index. Most people will only use one subaddress, and can default to 0. Exchanges or users who want to generate lots of new public addresses may use multiple subaddresses.

In [ ]:
subaddress_index = 0
client.get_balance(monitor_id_response.monitor_id, subaddress_index)

#### Send a Transaction

MobileCoin uses "request codes" to wrap public addresses. See below for how to generate request codes.

In [ ]:
address_code = "2nTy8m2VE5UMtfqRf12gEjZmFHKNTDEtNufQZNvE713ytYvdu2kqpbcncHJUSLwmgTCkB56Li9fsGwJF9LRYEQvoQCDzqVQEJETDNQKLzqHCzd"
target_address_response = client.parse_request_code(address_code)

# Construct the transaction
txo_list_response = client.get_unspent_tx_output_list(monitor_id_response.monitor_id, subaddress_index)
outlays = [{
    'value': 10, 
    'receiver': target_address_response.receiver
}]
tx_proposal_response = client.generate_tx(
    monitor_id_response.monitor_id, 
    subaddress_index, 
    txo_list_response.output_list, 
    outlays
)

# Send the transaction to consensus validators
client.submit_tx(tx_proposal_response.tx_proposal)

#### Public Address (Request Code)

In [ ]:
public_address_response = client.get_public_address(monitor_id_response.monitor_id, subaddress_index)
request_code_response = client.create_request_code(public_address_response.public_address)
print(f"Request code = {request_code_response}")